In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import datetime, time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/logistics-shopee-code-league/delivery_orders_march.csv
/kaggle/input/logistics-shopee-code-league/SLA_matrix.xlsx


In [2]:
# output = pd.read_csv("output.csv")

In [3]:
# output.to_csv('output_final.csv', columns = ['orderid','is_late'], index=False)

In [4]:
df = pd.read_csv("/kaggle/input/logistics-shopee-code-league/delivery_orders_march.csv")

In [5]:
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [6]:
# df_test = df.loc[0:30]
df_test = df

In [7]:
df_test.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [8]:
def convert_to_date(seconds):
    return int(time.strftime('%d', time.localtime(seconds)))

In [9]:
def convert_to_seconds(day, year = 2020, month = 3, hour = 0, minute = 0):
    t = datetime.datetime(year, month, day, hour, minute)
    return time.mktime(t.timetuple())

In [10]:
holidaylist = []
holidaylist.append(convert_to_seconds(day = 8))
holidaylist.append(convert_to_seconds(day = 25))
holidaylist.append(convert_to_seconds(day = 30))
holidaylist.append(convert_to_seconds(day = 31))
holidaylist.append(convert_to_seconds(day = 15))
holidaylist.append(convert_to_seconds(day = 22))
holidaylist.append(convert_to_seconds(day = 29))
holidaylist.append(convert_to_seconds(day = 1))

print(holidaylist)

[1583625600.0, 1585094400.0, 1585526400.0, 1585612800.0, 1584230400.0, 1584835200.0, 1585440000.0, 1583020800.0]


In [11]:
def get_holidays(start, end):
    holidays = 0
    for holiday in holidaylist:
        if convert_to_date(start) <= convert_to_date(holiday) and convert_to_date(end) >= convert_to_date(holiday):
            holidays += 1
    return holidays

In [12]:
def add_second_attempt_days(row):
    if not math.isnan(row['2nd_deliver_attempt']):
        days = convert_to_date(row['2nd_deliver_attempt']) - convert_to_date(row['1st_deliver_attempt'])
#         days = math.floor(timetaken / 86400)
        
        holidays = get_holidays(row['1st_deliver_attempt'],row['2nd_deliver_attempt'])
        
        if days > (3 + holidays):
            return 1
        else:
            return 0
    else:
        return 0
    

In [13]:
def get_sla_days(row):
    
    clean_buyer = row['buyeraddress'].lower()
    clean_seller = row['selleraddress'].lower()
    
    if "metro manila" in clean_buyer:
        
        if "metro manila" in clean_seller:
            return 3
        
        elif "luzon" in clean_seller:
             return 5
            
        else:
            return 7
        
    
    elif "luzon" in clean_buyer:
        if "metro manila" in clean_seller:
            return 5
        
        elif "luzon" in clean_seller:
             return 5
            
        else:
            return 7
    
    elif "visays" in clean_buyer:
        return 7
    
    else:
        return 7

In [14]:
def add_first_attempt_days(row):
    days = convert_to_date(row['1st_deliver_attempt']) - convert_to_date(row['pick'])
#     timetaken = row['1st_deliver_attempt'] - row['pick']
#     days = math.floor(timetaken / 86400)
    
    holidays = get_holidays(row['pick'],row['1st_deliver_attempt'])
    
    if days > (row['sla'] + holidays): 
        return 1
    else:
        return 0

    

In [15]:
def islate(row):
    if any([row["isfirstlate"],row["issecondlate"]]):
        return 1
    else:
        return 0


In [16]:
# df['sla'] = df.apply(get_sla_days, axis=1)

In [ ]:
df_test['sla'] = df_test.apply(get_sla_days, axis=1)

In [ ]:
df_test['issecondlate'] = df_test.apply(add_second_attempt_days, axis=1)

In [ ]:
df_test['isfirstlate'] = df_test.apply(add_first_attempt_days, axis=1)

In [ ]:
df_test['is_late'] = df_test.apply(islate, axis=1)

In [ ]:
# df_test.loc[:]

In [ ]:
df_test.to_csv('output.csv', columns = ['orderid','is_late'], index=False)